## Segmenting and Clustering Neighborhoods in Toronto

In [54]:
import requests # library to handle requests
from bs4 import BeautifulSoup
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library


In [55]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641'
data  = requests.get(url).text
soup = BeautifulSoup(data,"html5lib")

In [56]:
#soup

In [57]:
table=soup.find('table')
#table

In [58]:
table.findAll('td')
table.findAll('td')[7].text

'North York'

In [113]:
table_contents=pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])
table=soup.find('table')
tabledata = table.findAll('td')
for row in range(len(tabledata)):
    cell = {}
    if len(tabledata[row].text)==3:
        if tabledata[row].text in table_contents['PostalCode'].values:
            x = table_contents[table_contents['PostalCode'] == tabledata[row].text].index
            table_contents.iloc[x, 2] = table_contents.iloc[x, 2].values[0]+', '+tabledata[row+2].text
            
        else:
            table_contents = table_contents.append({'PostalCode':tabledata[row].text, 'Borough':tabledata[row+1].text,
                                                   'Neighborhood':tabledata[row+2].text}, ignore_index=True)
    else:
        pass
df=pd.DataFrame(table_contents)

In [114]:
df = df.replace('\n', '', regex=True)
df = df.replace('Not assigned', np.nan, regex=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [119]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M9A,Etobicoke,Islington Avenue


In [116]:
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [118]:
#df.head()
df.shape

(102, 3)